In [49]:
!pip install gensim

In [50]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [51]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

In [52]:
vec_king[:10]

array([ 0.12597656,  0.02978516,  0.00860596,  0.13964844, -0.02563477,
       -0.03613281,  0.11181641, -0.19824219,  0.05126953,  0.36328125],
      dtype=float32)

In [53]:
import pandas as pd
messages=pd.read_csv('SMSSpamCollection.txt',
                    sep='\t',names=["label","message"])

In [54]:
messages.shape

(5572, 2)

In [55]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [56]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrey_98/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [58]:
corpus[:2]

['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat',
 'ok lar joking wif u oni']

In [59]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [60]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))


In [61]:
words[:2]

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni']]

In [63]:
import gensim

In [64]:
## Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words)

In [65]:
model.corpus_count

5569

In [66]:
model.epochs

5

In [67]:
model.wv.similar_by_word('good')

[('my', 0.9986546635627747),
 ('happy', 0.9984652996063232),
 ('day', 0.9983446598052979),
 ('morning', 0.9982134103775024),
 ('all', 0.9982128143310547),
 ('where', 0.9981928467750549),
 ('night', 0.998185396194458),
 ('not', 0.9981766939163208),
 ('well', 0.9981682300567627),
 ('wa', 0.9981035590171814)]

In [68]:
model.wv['good'].shape

(100,)

In [69]:
words[:1]

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat']]

In [70]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [71]:
!pip install tqdm

In [72]:
from tqdm import tqdm

In [78]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|████████████████████████████████████| 5569/5569 [00:00<00:00, 17612.34it/s]


In [79]:
len(X)

5569

In [80]:
messages.shape

(5572, 2)

In [76]:
X[1]

array([-0.16578935,  0.1791723 ,  0.1130751 ,  0.08315537,  0.07789968,
       -0.4273434 ,  0.13180745,  0.37494206, -0.230911  , -0.11584368,
       -0.14565405, -0.33317983, -0.03926856,  0.0690046 ,  0.20108469,
       -0.14224753,  0.06473842, -0.28320706, -0.02574972, -0.45377067,
        0.1694125 ,  0.08256179,  0.07609369, -0.15751237, -0.02761158,
       -0.00994881, -0.15773393, -0.18920907, -0.21986803,  0.04817484,
        0.26553243,  0.01362802,  0.08939889, -0.15862127, -0.07532104,
        0.3643058 ,  0.05899221, -0.09540585, -0.11530045, -0.39703584,
        0.0904272 , -0.20805481, -0.15688802, -0.01456763,  0.12378065,
       -0.02768737, -0.12287173, -0.01937274,  0.20306066,  0.09610365,
        0.138952  , -0.16232422, -0.01771149,  0.05643875, -0.07464506,
        0.07805735,  0.15010054,  0.0161264 , -0.35946435,  0.12639256,
        0.01659738,  0.13378355, -0.01165696, -0.07040858, -0.2729501 ,
        0.2087234 ,  0.0553378 ,  0.1994466 , -0.27656716,  0.34

In [83]:
## Dependent Features
## Output Features
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

In [84]:
y.shape

(5569,)

In [85]:
X[0].reshape(1,-1).shape

(1, 100)

In [87]:
## this is the final independent features
import pandas as pd

# Assuming X is a list of numpy arrays
dfs = [pd.DataFrame(X[i].reshape(1, -1)) for i in range(len(X))]
df = pd.concat(dfs, ignore_index=True)


/var/folders/l2/8f2r1rwx6lx76vv6n6ncb1k40000gn/T/ipykernel_53601/357844545.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, ignore_index=True)


In [88]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.182639,0.208242,0.135395,0.095533,0.082732,-0.499929,0.170139,0.429740,-0.269640,-0.144885,...,0.346806,0.192415,0.026301,0.052786,0.435655,0.186245,0.168710,-0.212550,0.179083,-0.005921
1,-0.165789,0.179172,0.113075,0.083155,0.077900,-0.427343,0.131807,0.374942,-0.230911,-0.115844,...,0.300521,0.156280,0.016107,0.038208,0.359144,0.155205,0.143959,-0.190706,0.157416,-0.011291
2,-0.191308,0.222036,0.146309,0.109131,0.073037,-0.540101,0.167136,0.408618,-0.279875,-0.166972,...,0.346190,0.204998,0.014374,0.023922,0.444901,0.170915,0.123100,-0.242632,0.203992,0.021933
3,-0.249937,0.279024,0.174360,0.131232,0.112394,-0.669033,0.217307,0.585488,-0.366752,-0.184901,...,0.464889,0.251012,0.029540,0.076192,0.569226,0.254041,0.240053,-0.289830,0.237916,-0.017203
4,-0.217537,0.231340,0.155360,0.108782,0.103766,-0.567698,0.184006,0.500435,-0.313018,-0.165169,...,0.401234,0.210512,0.030325,0.067508,0.485411,0.218975,0.196292,-0.257395,0.196076,-0.013062


In [89]:
df['Output']=y

In [90]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.182639,0.208242,0.135395,0.095533,0.082732,-0.499929,0.170139,0.429740,-0.269640,-0.144885,...,0.192415,0.026301,0.052786,0.435655,0.186245,0.168710,-0.212550,0.179083,-0.005921,True
1,-0.165789,0.179172,0.113075,0.083155,0.077900,-0.427343,0.131807,0.374942,-0.230911,-0.115844,...,0.156280,0.016107,0.038208,0.359144,0.155205,0.143959,-0.190706,0.157416,-0.011291,True
2,-0.191308,0.222036,0.146309,0.109131,0.073037,-0.540101,0.167136,0.408618,-0.279875,-0.166972,...,0.204998,0.014374,0.023922,0.444901,0.170915,0.123100,-0.242632,0.203992,0.021933,False
3,-0.249937,0.279024,0.174360,0.131232,0.112394,-0.669033,0.217307,0.585488,-0.366752,-0.184901,...,0.251012,0.029540,0.076192,0.569226,0.254041,0.240053,-0.289830,0.237916,-0.017203,True
4,-0.217537,0.231340,0.155360,0.108782,0.103766,-0.567698,0.184006,0.500435,-0.313018,-0.165169,...,0.210512,0.030325,0.067508,0.485411,0.218975,0.196292,-0.257395,0.196076,-0.013062,True


In [91]:
df.dropna(inplace=True)

In [92]:
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [93]:
## Independent Feature
X=df

In [94]:
X.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [95]:
y=df['Output']

In [96]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [97]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
599,-0.246343,0.260158,0.167681,0.113350,0.105309,-0.628881,0.206645,0.552636,-0.339653,-0.181188,...,0.234581,0.034768,0.077391,0.540700,0.253995,0.228652,-0.266944,0.212716,-0.013948,True
689,-0.209779,0.229642,0.151283,0.113177,0.089318,-0.550505,0.174693,0.472586,-0.293418,-0.155747,...,0.205031,0.020479,0.061909,0.472904,0.206243,0.181072,-0.237044,0.185336,-0.000202,True
350,-0.234882,0.253937,0.166043,0.108172,0.102404,-0.606871,0.201125,0.530884,-0.325910,-0.186044,...,0.221674,0.033519,0.074832,0.514404,0.233430,0.212140,-0.258206,0.201427,-0.014050,True
5197,-0.191215,0.247858,0.164465,0.119948,0.073285,-0.575585,0.188980,0.438213,-0.299606,-0.178637,...,0.220655,0.019222,0.023759,0.483450,0.172180,0.136186,-0.246332,0.217861,0.015643,False
440,-0.259906,0.275230,0.183802,0.130858,0.120496,-0.676596,0.207785,0.580093,-0.362867,-0.203300,...,0.245650,0.030929,0.076124,0.571083,0.253089,0.208359,-0.299763,0.225290,0.003642,True


In [98]:
y_train

599      True
689      True
350      True
5197    False
440      True
        ...  
4222     True
4430     True
3903    False
46       True
3157     True
Name: Output, Length: 4445, dtype: bool

In [103]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [104]:
X_train.columns = X_train.columns.astype(str)


In [105]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [107]:
X_test.columns = X_test.columns.astype(str)

In [108]:
y_pred=classifier.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.9991007194244604


In [110]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       1.00      0.99      1.00       165
        True       1.00      1.00      1.00       947

    accuracy                           1.00      1112
   macro avg       1.00      1.00      1.00      1112
weighted avg       1.00      1.00      1.00      1112

